# TensorBoard 效能調校

In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist


# 匯入 MNIST 手寫阿拉伯數字 訓練資料
(x_train, y_train),(x_test, y_test) = mnist.load_data()


# 訓練/測試資料的 X/y 維度
x_train.shape, y_train.shape,x_test.shape, y_test.shape

# 特徵縮放，使用常態化(Normalization)，公式 = (x - min) / (max - min)
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0


In [6]:
# 建立、訓練、評估模型
def train_test_model(hparams):
    # 建立模型
    model = tf.keras.models.Sequential([
      tf.keras.layers.Input((28, 28)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    # 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    # 訓練
    model.fit(x_train_norm, y_train, epochs=5, validation_split=0.2)
    # 評估，打分數
    _, accuracy = model.evaluate(x_test_norm, y_test, verbose=0)
    return accuracy

In [7]:
# 定義訓練函數，包括依參數訓練並寫入 log
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)    # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar('accuracy', accuracy, step=1)

In [8]:
# 參數組合
from tensorboard.plugins.hparams import api as hp

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

In [9]:
# 依每一參數組合執行訓練
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                    HP_NUM_UNITS: num_units,
                    HP_DROPOUT: dropout_rate,
                    HP_OPTIMIZER: optimizer,
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8478 - loss: 0.5267 - val_accuracy: 0.9538 - val_loss: 0.1571
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9512 - loss: 0.1663 - val_accuracy: 0.9643 - val_loss: 0.1182
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9620 - loss: 0.1270 - val_accuracy: 0.9713 - val_loss: 0.0976
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9699 - loss: 0.0953 - val_accuracy: 0.9721 - val_loss: 0.0898
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9743 - loss: 0.0813 - val_accuracy: 0.9762 - val_loss: 0.0843
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8384 - loss: 0.5476 - val_accuracy: 0.9547 - val_loss: 0.1600
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 

In [10]:
# 載入 TensorBoard notebook extension，即可在 jupyter notebook 啟動 Tensorboard
%load_ext tensorboard

In [12]:
# 啟動 Tensorboard
%tensorboard --logdir logs/hparam_tuning

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 5524.

In [13]:
!taskkill /IM "tensorboard.exe" /F
# or 
# !taskkill /F /PID 15784

成功: 處理程序 "tensorboard.exe" (PID 5524) 已經終止了。
